In [3]:
#LOIS NOTEBOOK
import pandas as pd

In [5]:
#datasets from ml-25m
movies_df = pd.read_csv('./datasets/ml-25m/movies.csv')
ratings_df = pd.read_csv('./datasets/ml-25m/ratings.csv')
links_df = pd.read_csv('./datasets/ml-25m/links.csv')
tags_df = pd.read_csv('./datasets/ml-25m/tags.csv')
genome_scores_df = pd.read_csv('./datasets/ml-25m/genome-scores.csv')
genome_tags_df = pd.read_csv('./datasets/ml-25m/genome-tags.csv')

FileNotFoundError: [Errno 2] No such file or directory: './datasets/ml-25m/movies.csv'

In [90]:
#creation of main table for machine learning

#groupby ratings_df to get the average ratings per movie
ratings_df = ratings_df.groupby(by=["movieId"]).mean()

#split genres by |
movies_df['genres_split'] = movies_df['genres'].str.split('|').apply(lambda x: ', '.join(x))
merged_df = pd.merge(movies_df, ratings_df, on='movieId', how='left')
tags_df['tag'] = tags_df['tag'].astype(str)
tags_grouped = tags_df.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
merged_df = pd.merge(merged_df, tags_grouped, on='movieId', how='left')

In [92]:
final_movies_df = merged_df[['movieId', 'title', 'genres_split', 'rating', 'tag']]
final_movies_df = final_movies_df.rename(columns={'movieId': 'movie_id', 'userId': 'user_id'})

data = final_movies_df

In [93]:
data['tags_genres'] = data['tag']+data['genres_split']
data_for_processing  = data.drop(columns=['tag', 'genres_split'])
data_for_processing.head()

,movie_id,title,rating,tags_genres
0,1,Toy Story (1995),3.893708,Owned imdb top 250 Pixar Pixar time travel chi...
1,2,Jumanji (1995),3.251527,Robin Williams time travel fantasy based on ch...
2,3,Grumpier Old Men (1995),3.142028,funny best friend duringcreditsstinger fishing...
3,4,Waiting to Exhale (1995),2.853547,based on novel or book chick flick divorce int...
4,5,Father of the Bride Part II (1995),3.058434,aging baby confidence contraception daughter g...


In [98]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000, stop_words='english')
vector=cv.fit_transform(data_for_processing['tags_genres'].values.astype('U')).toarray()
vector.shape

(62423, 10000)

In [99]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)
similarity

array([[1.        , 0.0811537 , 0.04219512, ..., 0.        , 0.        ,
        0.        ],
       [0.0811537 , 1.        , 0.00174817, ..., 0.        , 0.        ,
        0.        ],
       [0.04219512, 0.00174817, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ]])

In [101]:
data_for_processing[data_for_processing['title']=="Toy Story (1995)"].index[0]


0

In [103]:
distance = sorted(list(enumerate(similarity[2])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:5]:
    print(data_for_processing.iloc[i[0]].title)

Grumpier Old Men (1995)
Grumpy Old Men (1993)
The Book of Daniel (2013)
Pictures of the Old World (Obrazy starého sveta) (1972)
Lydia (1941)


In [107]:
def recommend(movies):
    index=data_for_processing[data_for_processing['title']==movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[0:5]:
        print(data_for_processing.iloc[i[0]].title)

In [108]:
recommend("Toy Story (1995)")

Toy Story (1995)
Toy Story 2 (1999)
Bug's Life, A (1998)
Monsters, Inc. (2001)
Toy Story 3 (2010)


In [116]:
recommend("Ju-on: The Grudge (2002)")

Ju-on: The Grudge (2002)
Ring, The (2002)
Faces of Death 2 (1981)
Faces of Death 3 (1985)
Night of the Day of the Dawn of the Son of the Bride of the Return of the Terror (1991)


In [1]:
#organize movie titles & ratings of one user into text
merged_df = pd.merge(ratings_df, movies_df[['movieId', 'title']], on='movieId', how='left')

def get_user_latest_rating(user_id):
    user_ratings = merged_df[merged_df['userId'] == user_id]
    user_ratings = user_ratings.sort_values('timestamp', ascending=False)
    return user_ratings.head(10)

# Test run of get_user_latest_rating function
user_id = 1
latest_ratings = get_user_latest_rating(user_id)
# print(latest_ratings)

# Convert latest_ratings to JSON for chatgpt
latest_ratings_json = latest_ratings.to_json(orient='records')
with open('latest_ratings.json', 'w') as file:
    file.write(latest_ratings_json)

NameError: name 'ratings_df' is not defined